In [29]:
import pandas as pd
import numpy as np

In [30]:
raw = pd.read_csv('http://hilpisch.com/pyalgo_eikon_eod_data.csv',
                           index_col=0, parse_dates=True).dropna()

In [31]:
data = pd.DataFrame(raw['EUR='])

In [32]:
data.rename(columns={'EUR=': 'price'}, inplace=True)

In [33]:
lags = 5

In [35]:
cols = []
for lag in range(1, lags + 1):
    col = f'lag_{lag}'
    data[col] = data['price'].shift(lag)
    cols.append(col)
    data.dropna(inplace=True)

In [36]:
reg = np.linalg.lstsq(data[cols], data['price'],
                               rcond=None)[0]

In [37]:
reg

array([ 0.98243806,  0.02602424, -0.04776545,  0.05253327, -0.01333633])